In [16]:
import os
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import re
import urllib.request

# MBTI 별 posting 모으기

In [17]:
mbti=pd.read_csv('./data/mbti_1.csv')

types = mbti.groupby('type').count()

output_file = '.\data\\training\\mbti.csv' # raw string이 아니라 '\'를 쓰려면 \\라고 해야 함
allData = []

for type in types.index:
    condition = mbti['type'] == type # condition: mbti['type']가 topfive의 원소인 type 같을 bool 조건
    ownsentence = mbti[condition]  # ownsentence: condition에 맞는 row만 filtering한 dataframe
    allData.append(ownsentence)
dataCombine = pd.concat(allData, axis=0, ignore_index=True)
dataCombine.to_csv(output_file, index=False)

types

    #input_file = r'.\data\training'
    #allFile_list = glob.glob(os.path.join(input_file, 'mbti_*'))
    #for file in allFile_list:
    #    csv = pd.read_csv(file,sep=';', encoding='iso-8859-1') # for구문으로 csv파일들을 읽어 들인다
    #    cleanMbti = csv['posts'].str.replace('[^A-Za-z\s]+', '')

,posts
type,
ENFJ,190
ENFP,675
ENTJ,231
ENTP,685
ESFJ,42
ESFP,48
ESTJ,39
ESTP,89
INFJ,1470


# Resampling (Undersampling and Oversampling)

# Tokenize and Stemming
morph analysis 기법.

In [18]:
import nltk
import gc
# nltk.download('punkt')
# nltk.download('wordnet')
from nltk import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

In [19]:
mbti= dataCombine
X_data = mbti['posts']
y_data = mbti['type']

normalized_text = []
for sentence in X_data.tolist():
    clean_sentence = re.sub('[^A-Za-z\s]+', '',sentence.lower())    # posts 데이터 정규화()
    normalized_text.append(clean_sentence)
mbti.posts = normalized_text

In [20]:
ps=PorterStemmer()

# print("stopwords 제거 시작")
stopwords = ['http*']
#print("stopwords 제거 완료")

stemData=[] # stemmed X_data: mbti['posts']
tokenizedData=[]    # tokenized X_data
for sentence in mbti['posts']:
    tokenData = nltk.word_tokenize(sentence)
    tempData = []   # 개별 post
    for word in tokenData:
        tempData.append(ps.stem(word))
        #tempData = [word for word in tempData if not word in stopwords] # 불용어 제거
    tokenizedData.append(tokenData)
    stemData.append(tempData)
#print(stemData[0])

#'list' object has no attribute 'lower' 문제 발생
# 2차원 리스트인 stemData를 1차원 리스트로
#flat_stem = [item for sublist in stemData for item in sublist]

del [[mbti]]
gc.collect()

21957

# Data 확인
실제 data와 stemmed Data의 양상이 일치하는지 확인. 확인 결과 한 post의 길이가 지나치게 적은 곳이 있었지만, 결과에 유의미하게 부정적인 영향을 미치지 않는다고 판단해서 정제하지 않기로 했다.

In [ ]:
# bagoftoken = tokenizedData.flatten()
# bagofstemmed = stemData.flatten()

print('post의 최대 길이(단어 수) :{}'.format(max(len(l) for l in stemData)))
print('post의 최소 길이 :{}'.format(min(len(l) for l in stemData)))
print('post의 평균 길이 :{}'.format(sum(map(len, stemData))/len(stemData)))
print('post의 전체 길이 :{}'.format(sum(map(len, stemData))))
print('post의 갯수: {}'.format(len(stemData)))
print('첫 post의 단어 개수:{}'.format(len(stemData[0])))

plt.hist([len(s) for s in stemData], bins=50)
plt.xlabel('length of stemmed posts')
plt.ylabel('number of stemmed posts')
plt.show()

plt.hist([len(s) for s in tokenizedData], bins=50)
plt.xlabel('length of raw posts')
plt.ylabel('number of raw posts')
plt.show()

# Vectorization(tfidf)
padding 과정이 따로 필요하지 않은 output이 나와서 padding 생략

In [7]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

In [8]:


def dummy_fun(doc):
    return doc

tfidf=TfidfVectorizer(
    analyzer='word',
    tokenizer=dummy_fun,
    preprocessor=dummy_fun,
    token_pattern=None
)

X = tfidf.fit_transform(stemData).toarray()
print(X)

print(y_data)
y = np.array(y_data)
print(y)

# le = LabelEncoder() # string인 'type' label을 int로 바꿔준다 (input: array)
# y = le.fit_transform(y).reshape(-1,1)
# y = to_categorical(y)
'''tf = TfidfVectorizer().fit(stemData)
print(tf.transform(stemData).toarray())
print(tf.vocabulary_)
#print(vector.vocabulary_)'''

[[0.15619188 0.         0.         ... 0.         0.         0.        ]
 [0.22128246 0.04019078 0.         ... 0.         0.         0.        ]
 [0.14145265 0.         0.         ... 0.         0.         0.        ]
 ...
 [0.18796687 0.         0.         ... 0.         0.         0.        ]
 [0.19099817 0.         0.         ... 0.         0.         0.        ]
 [0.13217717 0.         0.         ... 0.         0.         0.        ]]
0       ENFJ
1       ENFJ
2       ENFJ
3       ENFJ
4       ENFJ
        ... 
8670    ISTP
8671    ISTP
8672    ISTP
8673    ISTP
8674    ISTP
Name: type, Length: 8675, dtype: object
['ENFJ' 'ENFJ' 'ENFJ' ... 'ISTP' 'ISTP' 'ISTP']


'tf = TfidfVectorizer().fit(stemData)\nprint(tf.transform(stemData).toarray())\nprint(tf.vocabulary_)\n#print(vector.vocabulary_)'

# Garbage Collection

In [9]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from matplotlib import pyplot

In [10]:
print("Model building Start")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.3, random_state=1234)
model = XGBClassifier()
model.fit(X_train, y_train)


Model building Start


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [11]:
preds = model.predict(X_test)
print("Model building End")

Model building End


In [13]:
import joblib

joblib.dump(model, 'XGBoost_rawsample.pkl')

['XGBoost_rawsample.pkl']

# Evaluation

In [28]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from yellowbrick.classifier import ROCAUC

print("Evaluation Start")
# labels과 guesses
labels = preds
guesses = y_test

print(accuracy_score(labels, guesses))
print(recall_score(labels, guesses, average=None))
print(precision_score(labels, guesses, average=None))
print(f1_score(labels, guesses, average=None))

print("Evaluation END")
# https://eunsukimme.github.io/ml/2019/10/21/Accuracy-Recall-Precision-F1-score/

Evaluation Start
0.6496350364963503
[0.6        0.65306122 0.6875     0.64974619 1.         0.33333333
 0.66666667 0.4        0.64574899 0.65785381 0.65662651 0.64268585
 0.67647059 0.59090909 0.66666667 0.68235294]
[0.36       0.61835749 0.5        0.59813084 0.08333333 0.06666667
 0.25       0.26086957 0.71205357 0.76909091 0.68125    0.67848101
 0.43396226 0.45348837 0.39393939 0.64444444]
[0.45       0.63523573 0.57894737 0.62287105 0.15384615 0.11111111
 0.36363636 0.31578947 0.67728238 0.70913663 0.66871166 0.66009852
 0.52873563 0.51315789 0.4952381  0.66285714]
Evaluation END
